## Basic imports 

In [ ]:
import sys 
sys.path.append("../")
import barry 
import numpy as np
import matplotlib.pyplot as plt 

## Plot power spectrum with varying Neff 
## - this is to verify Barry is able to get power spectrum with varying Neff 

In [ ]:
from barry.models import PowerBeutler2017
#from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4

# Set up the appropriate model for the power spectrum

model = PowerBeutler2017(
    vary_neff=True,
    isotropic=True,
    n_poly=6,
) # setting up choice for BAO model 

cosmology = {
            "om": 0.31,
            "h0": 0.67,
            "z":  0.51,
            "ob": 0.048,
            "ns": 0.97,
            "mnu": 0.0,
            "reconsmoothscale": 15,
            "Neff": 3.044,
        }
model.set_cosmology(cosmology) # initialises self.camb object 
k = model.camb.ks

# model.camb._generate_data() # - only needs to be run one time to calculate power spectrum at varying Neff 


ax1 = plt.subplot(121)
ax2 = plt.subplot(122)

#ax1.set_xlim(2e-4, 100)
#ax2.set_xlim(2e-4, 100)

ax1.set_ylim(5e-5, 1e6)
ax2.set_ylim(-0.5, 3)


ax1.set_title("Matter power spectrum")
ax2.set_title(r"Matter power spectrum ratio to $N_{\mathrm{eff}}$=3.044 model")

pk_3044 = model.camb.get_data(cosmology["om"], Neff=3.044)

for i in [0.0, 1.0, 2.0, 3.0, 4.0, 5.0]:
    
    # model._generate_data(cosmology["om"], Neff=i)
    pk = model.camb.get_data(cosmology["om"], Neff=i) # interpolates for desired Neff and returns power spectrum data 
    # (only works as long as self.cab._generate_data() has been run) 
    
    ax1.loglog(k, pk['pk_lin_0'], label=r"$N_{\mathrm{eff}}$ = %.2f" % i)
    
    ax2.semilogx(k, pk['pk_lin_0']/pk_3044['pk_lin_0'], label="$N_{\mathrm{eff}}$ = %.2f" % i)
    
plt.subplots_adjust(wspace=0.4, left=0, right=1.5)    
ax2.set_ylim([0.5, 2.5])
ax1.legend()
ax2.legend()




In [ ]:
from barry.models import PowerBeutler2017
#from barry.datasets.dataset_power_spectrum import PowerSpectrum_DESI_KP4

# Set up the appropriate model for the power spectrum

model = PowerBeutler2017(
    vary_neff=True,
) # setting up choice for BAO model 

cosmology = {
            "om": 0.31,
            "h0": 0.67,
            "z":  0.51,
            "ob": 0.048,
            "ns": 0.97,
            "mnu": 0.0,
            "reconsmoothscale": 15,
            "Neff": 3.044,
        }

model.set_cosmology(cosmology) # initialises self.camb object 
# model.camb._generate_data()
k = model.camb.ks


pk_306 = model.camb.get_data(cosmology["om"], Neff=3.044+0.02)
pk_303 = model.camb.get_data(cosmology["om"], Neff=3.044-0.02)

neff_delta= 0.04

plt.semilogx(k, (pk_306['pk_lin_0'] - pk_303['pk_lin_0'])/neff_delta) 
plt.xlim([1e-3, 1])
plt.show() 

## Plot the BAO model for 
## - smoothed power spectrum 
## - wiggles 
## with the varying choices of Neff 

In [ ]:
ax1 = plt.subplot(121)
ax2 = plt.subplot(122)

ax2.set_xlim(1e-4, 100)
ax1.set_xlim(1e-4, 100)

ax2.set_ylim(-0.1, 0.1)
ax1.set_ylim(1e-3, 1e5)


for i in [2.0, 3.0, 4.0, 5.0]:
    
    pksmooth, pkwiggles = model.compute_basic_power_spectrum(0.31, Neff=i) # interpolates for desired Neff and returns power spectrum data 
    # (only works as long as self.cab._generate_data() has been run) d

    ax1.loglog(k, pksmooth, label=r"$N_{\mathrm{eff}}$ = %.2f" % i)
    ax2.semilogx(k, pkwiggles, label=r"$N_{\mathrm{eff}}$= %.2f" % i)
    
ax1.set_title('Smoothed power spectrum')
ax2.set_title('Power spectrum wiggles')
ax2.set_xlim(5e-4, 0.5)
plt.subplots_adjust(wspace=0.2, left=0, right=1.8)   
ax1.legend()
ax2.legend()

## Plot the full power spectrum model moments for the BAOs, with Neff varying for generating different cosmologies 

In [ ]:
import sys 
sys.path.append("../")
import barry 
import numpy as np
import matplotlib.pyplot as plt 
from barry.models import PowerBeutler2017

model = PowerBeutler2017(
    vary_neff=True,
    isotropic=True,
    #poly_poles=[0],
    n_poly=6,
) # setting up choice for BAO model - now isotropic model for plotting 

cosmology = {
            "om": 0.31,
            "h0": 0.67,
            "z":  0.001,
            "ob": 0.048,
            "ns": 0.97,
            "mnu": 0.06,
            "reconsmoothscale": 15,
            "Neff": 3.044,
        }
model.set_cosmology(cosmology) # initialises self.camb object 
ks = model.camb.ks
params_dict = {j.name: j.default for j in model.params}

ax1 = plt.subplot(121)
ax2 = plt.subplot(122)

params_dict["Neff"] = 3.044
k, pk1 = model.compute_power_spectrum(ks, params_dict, smooth=True)

for neff in [2.0, 3.0, 3.044, 4.0, 5.0]:

    model.pksmooth = None 
    model.pkratio = None
    
    params_dict["Neff"] = neff
    k, pk = model.compute_power_spectrum(ks, params_dict, vary_neff=True)
    
    ax1.loglog(k, pk[0], label=r"$N_{\mathrm{eff}}$ = %.3f" % neff)
    ax2.semilogx(k, pk[0]/pk1[0], label=r"$N_{\mathrm{eff}}$ = %.3f" % neff)
    
    

print("Beutler et al 2017 power spectrum model, recomputed with varying Neff")
ax1.set_title(r'isotropic monopole, $\mu = 0$')
ax2.set_title(r'ratio isotropic monopole to smooth model with $N_{\mathrm{eff}}$=3.044, $\mu = 0$')
plt.subplots_adjust(wspace=0.6, left=0, right=2)
ax1.legend()
plt.show()

In [ ]:
import sys 
sys.path.append("../")
import barry 
import numpy as np
import matplotlib.pyplot as plt 

from barry.models import PowerBeutler2017

model = PowerBeutler2017(
    vary_neff=True,
    isotropic=False,
    #poly_poles=[0],
    n_poly=6,
) # setting up choice for BAO model - now anisotropic model for plotting 

cosmology = {
            "om": 0.31,
            "h0": 0.67,
            "z":  0.001,
            "ob": 0.048,
            "ns": 0.97,
            "mnu": 0.06,
            "reconsmoothscale": 15,
            "Neff": 3.044,
        }
model.set_cosmology(cosmology) # initialises self.camb object 
ks = model.camb.ks
params_dict = {j.name: j.default for j in model.params}
params_dict['beta'] = 0.525

ax1 = plt.subplot(131)
ax2 = plt.subplot(132)
ax3 = plt.subplot(133)

for neff in [2.0, 3.0, 4.0, 5.0]:

    model.pksmooth = None 
    model.pkratio = None
    
    params_dict["Neff"] = neff
    k, pk = model.compute_power_spectrum(ks, params_dict, vary_neff=True)
    
    ax1.loglog(k[:,0], pk[0], label=r"$N_{\mathrm{eff}}$ = %.3f" % neff)
    ax2.semilogx(k[:,0], pk[2], label=r"$N_{\mathrm{eff}}$ = %.3f" % neff)
    ax3.semilogx(k[:,0], pk[4], label=r"$N_{\mathrm{eff}}$ = %.3f" % neff)
    

print("Beutler et al 2017 power spectrum model, recomputed with varying Neff")
ax1.set_title(r'anisotropic monopole, $\mu = 0$')
ax2.set_title(r'anisotropic quadrupole, $\mu = 0$')
ax3.set_title(r'anisotropic hexadecapole, $\mu = 0$')
plt.subplots_adjust(wspace=0.6, left=0, right=2)
ax1.legend()
plt.show()

## Keep a constant value for Neff when generating Beutler 2017 power spectrum model with CAMB but vary the phase shift parameter for Neff and plot the models

In [ ]:
import sys 
sys.path.append("../")
#sys.path.append("../../")
import barry 
import numpy as np
import matplotlib.pyplot as plt 

from barry.models import PowerBeutler2017

model = PowerBeutler2017(
    isotropic=True,
    vary_phase_shift_neff=True,
    #poly_poles=[0],
    dilate_smooth=False,
    n_poly=6,
) # setting up choice for BAO model - isotropic model for plotting 

cosmology = {
            "om": 0.31,
            "h0": 0.67,
            "z":  0.51,
            "ob": 0.048,
            "ns": 0.97,
            "mnu": 0.0,
            "reconsmoothscale": 15,
            "Neff": 3.044,
        }
model.set_cosmology(cosmology) # initialises self.camb object 
ks = model.camb.ks
params_dict = {j.name: j.default for j in model.params}

ax1 = plt.subplot(121)
ax2 = plt.subplot(122)

ax2.set_xlim(5e-4, 0.5)
ax1.set_xlim(5e-4, 10.0)

params_dict["beta_phase_shift"] = 1.0
k, pk1 = model.compute_power_spectrum(ks, params_dict, smooth=True)

for phase_shift_parameter in [0.5, 0.75, 1.0, 1.25, 1.5, 2.0, 2.5]:
#for a in [0.9, 0.95, 1.0, 1.05, 1.1]:  
#for e in [-0.1, -0.05, 0.0, 0.05, 0.1]:  
    
    params_dict["beta_phase_shift"] = phase_shift_parameter
    #params_dict["alpha"] = a
    #params_dict["epsilon"] = e 
    
    k, pk = model.compute_power_spectrum(ks, params_dict)
    
    ax1.loglog(k, pk[0], label=r"$\beta(N_{\mathrm{eff}})$ = %.3f" % phase_shift_parameter)#, alpha=0.7)
    ax2.semilogx(k, pk[0]/pk1[0], label=r"$\beta(N_{\mathrm{eff}})$ = %.3f" % phase_shift_parameter, alpha=0.9)
    
   

print("Beutler et al 2017 power spectrum model, recomputed with varying Neff phase shift parameter beta")
ax1.set_title('isotropic monopole')
ax2.set_title(r'ratio isotropic monopole to model with $N_{\mathrm{eff}}$=3.044 and no wiggles ($\beta$=1)')
plt.subplots_adjust(wspace=0.1, left=0, right=2, top=1)
ax1.legend()
ax2.legend()
plt.show() 

In [ ]:
import sys 
sys.path.append("../")
#import barry 
import numpy as np
import matplotlib.pyplot as plt 
from barry.models import PowerBeutler2017

model = PowerBeutler2017(
    isotropic=False,
    vary_phase_shift_neff=True,
    n_poly=6,
    dilate_smooth=True
) # setting up choice for BAO model - anisotropic model for plotting 

cosmology = {
            "om": 0.31,
            "h0": 0.67,
            "z":  0.51,
            "ob": 0.048,
            "ns": 0.97,
            "mnu": 0.0,
            "reconsmoothscale": 15,
            "Neff": 3.044,
        }
model.set_cosmology(cosmology) # initialises self.camb object 
ks = model.camb.ks
params_dict = {j.name: j.default for j in model.params}
params_dict['beta'] = 0.525


ax1 = plt.subplot(331)
ax2 = plt.subplot(332)
ax3 = plt.subplot(333)

ax4 = plt.subplot(334)
ax5 = plt.subplot(335)
ax6 = plt.subplot(336)

ax7 = plt.subplot(337)
ax8 = plt.subplot(338)
ax9 = plt.subplot(339)

params_dict["beta_phase_shift"] = 1.0
k, pkref = model.compute_power_spectrum(ks, params_dict, smooth=True)

for phase_shift_parameter in [0., 1.0, 2.0, 3.0]:
#for e in [-0.01, -0.005, 0.0, 0.005, 0.01]:  

    params_dict["beta_phase_shift"] = phase_shift_parameter
    #params_dict["epsilon"] = e 
    k, pk = model.compute_power_spectrum(ks, params_dict)
    
    # ax1.loglog(k[:,0], pk[0], label=r"$\beta(N_{\mathrm{eff}})$ = %.3f" % phase_shift_parameter)#, alpha=0.6)
    # ax2.semilogx(k[:,0], pk[2], label=r"$\beta(N_{\mathrm{eff}})$ = %.3f" % phase_shift_parameter)#, alpha=0.6)
    # ax3.semilogx(k[:,0], pk[4], label=r"$\beta(N_{\mathrm{eff}})$ = %.3f" % phase_shift_parameter)#, alpha=0.6)
    
    ax1.loglog(k[:,0], pk[0]/pkref[0], label=r"$\beta(N_{\mathrm{eff}})$ = %.3f, $\mu = 0$" % phase_shift_parameter)#, alpha=0.6)
    ax2.semilogx(k[:,0], pk[2]/pkref[2], label=r"$\beta(N_{\mathrm{eff}})$ = %.3f, $\mu = 0$" % phase_shift_parameter)#, alpha=0.6)
    ax3.semilogx(k[:,0], pk[4]/pkref[4], label=r"$\beta(N_{\mathrm{eff}})$ = %.3f, $\mu = 0$" % phase_shift_parameter)#, alpha=0.6)
    
    ax4.loglog(k[:,int(len(k[0,:])/2)], pk[0]/pkref[0], label=r"$\beta(N_{\mathrm{eff}})$ = %.3f, $\mu = 0.5$" % phase_shift_parameter)#, alpha=0.6)
    ax5.semilogx(k[:,int(len(k[0,:])/2)], pk[2]/pkref[2], label=r"$\beta(N_{\mathrm{eff}})$ = %.3f, $\mu = 0.5$" % phase_shift_parameter)#, alpha=0.6)
    ax6.semilogx(k[:,int(len(k[0,:])/2)], pk[4]/pkref[4], label=r"$\beta(N_{\mathrm{eff}})$ = %.3f, $\mu = 0.5$" % phase_shift_parameter)#, alpha=0.6)
    
    ax7.loglog(k[:,-1], pk[0]/pkref[0], label=r"$\beta(N_{\mathrm{eff}})$ = %.3f, $\mu = 1$" % phase_shift_parameter)#, alpha=0.6)
    ax8.semilogx(k[:,-1], pk[2]/pkref[2], label=r"$\beta(N_{\mathrm{eff}})$ = %.3f, $\mu = 1$" % phase_shift_parameter)#, alpha=0.6)
    ax9.semilogx(k[:,-1], pk[4]/pkref[4], label=r"$\beta(N_{\mathrm{eff}})$ = %.3f, $\mu = 1$" % phase_shift_parameter)#, alpha=0.6)
    
    


print(r'Beutler et al 2017 power spectrum model, recomputed with varying Neff phase shift parameter beta')

ax1.set_title(r'anisotropic monopole, $\mu = 0$, ratio to $\beta=1$')
ax2.set_title(r'anisotropic quadrupole, $\mu = 0$, ratio to $\beta=1$')
ax3.set_title(r'anisotropic hexadecapole, $\mu = 0$, ratio to $\beta=1$')

ax4.set_title(r'anisotropic monopole, $\mu = 0.5$, ratio to $\beta=1$')
ax5.set_title(r'anisotropic quadrupole, $\mu = 0.5$, ratio to $\beta=1$')
ax6.set_title(r'anisotropic hexadecapole, $\mu = 0.5$, ratio to $\beta=1$')

ax7.set_title(r'anisotropic monopole, $\mu = 1$, ratio to $\beta=1$')
ax8.set_title(r'anisotropic quadrupole, $\mu = 1$, ratio to $\beta=1$')
ax9.set_title(r'anisotropic hexadecapole, $\mu = 1$, ratio to $\beta=1$')

# ax4.set_title(r'ratio to smooth case with $\beta$=1, $\mu = 0$')
# ax5.set_title(r'ratio to smooth case with $\beta=1$, $\mu = 0$')
# ax6.set_title(r'ratio to smooth case with $\beta=1$, $\mu = 0$')

plt.subplots_adjust(wspace=0.2, hspace=0.4, left=0, right=2.25, bottom=-1.75)
ax1.legend(ncol=2)

ax1.set_xlim([0.5e-4, 2])
ax2.set_xlim([0.5e-4, 2])
ax3.set_xlim([0.5e-4, 2])

ax4.set_xlim([0.5e-4, 2])
ax5.set_xlim([0.5e-4, 2])
ax6.set_xlim([0.5e-4, 2])

ax7.set_xlim([0.5e-4, 2])
ax8.set_xlim([0.5e-4, 2])
ax9.set_xlim([0.5e-4, 2])

ax2.set_ylim([0.85,1.2])
ax3.set_ylim([0.85,1.2])
ax5.set_ylim([0.85,1.2])
ax6.set_ylim([0.85,1.2])
ax8.set_ylim([0.85,1.2])
ax9.set_ylim([0.85,1.2])

plt.show() 

## Testing code with CLASS instead of CAMB 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from classy import Class 
M = Class()

ob=0.04814
ns=0.97 
om = 0.31
h0 = 0.676
ks_fid = np.logspace(np.log10(0.001 * h0), np.log10(10.0 * h0), 100, base=10) # ks in 1/ MPC unit
f, (ax1, ax2) = plt.subplots(1,2)

###################################################################################################

arr = np.array([1.044, 2.044, 3.044, 4.044, 5.044, 6.044])

for i in arr:

    M.set({
        "omega_b": ob * h0 * h0, 
        "omega_cdm": (om-ob) * h0 * h0, 
        "H0": h0 * 100.0, 
        "A_s": 2.083e-9, 
        "N_ur": i, 
        "N_ncdm": 0.0,  
        "tau_reio": 0.066, 
        "n_s": ns
    })

    M.set({"output": "mPk", "P_k_max_1/Mpc": 10.0 * h0, "z_max_pk": 0.0})
    
    M.compute()
    
    Pk = np.array([M.pk_lin(ki, 0.0) /(h0 * h0 * h0) for ki in ks_fid]) 

    ax1.loglog(ks_fid, Pk, label = r'$N_{\mathrm{eff}}$ = %.3f' % M.Neff())

ax1.set_title(r'$N_{\mathrm{eff}} = \Delta N_{\mathrm{ur}}$')
ax1.legend()

###################################################################################################
arrn = np.array([1.0, 2.0, 3.0, 3.0, 3.0, 3.0, 3.0])
arr = np.array([0.031, 0.018, 0.0044, 1.0044, 2.0044, 3.0044])

for i in np.arange(len(arr)):

#     mass = '0.0'#'0.056'
#     if arrn[i] == 2:
#         mass = '0.0,0.0'#'0.028,0.028'
#     elif arrn[i] == 3:
#         mass = '0,0,0'#'0.01866, 0.01866, 0.01866'
        
    M.set({
        "omega_b": ob * h0 * h0, 
        "omega_cdm": (om-ob) * h0 * h0, 
        "H0": h0 * 100.0, 
        "A_s": 2.083e-9, 
        "N_ur": arr[i], 
        "N_ncdm": arrn[i],  
        # "m_ncdm": mass,
        "tau_reio": 0.066, 
        "n_s": ns
    })

    M.set({"output": "mPk", "P_k_max_1/Mpc": 10.0 * h0, "z_max_pk": 0.0})
    
    M.compute()
    
    Pk = np.array([M.pk_lin(ki, 0.0) /(h0 * h0 * h0) for ki in ks_fid]) 

    ax2.loglog(ks_fid, Pk, label = r'$N_{\mathrm{eff}}$ = %.3f' % M.Neff())
    
    #ax1.loglog(ks_fid, Pk, label = r'$N_{\mathrm{eff}}$ = %.3f' % M.Neff(), linestyle='--')
    
    
ax2.set_title(r'$N_{\mathrm{eff}} = N_{\nu} + \Delta N_{\mathrm{ur}}$')
ax2.legend()

###################################################################################################

plt.subplots_adjust(wspace=0.5, left=0.0, right=1.5)

plt.show()






In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from classy import Class 
M = Class()

ob=0.04814
ns=0.97 
om = 0.31
h0 = 0.676
ks_fid = np.logspace(np.log10(0.001 * h0), np.log10(10.0 * h0), 100, base=10) # ks in 1/ MPC unit

# power spectrum to normalize 

M.set({
        "omega_b": ob * h0 * h0, 
        "omega_cdm": (om-ob) * h0 * h0, 
        "H0": h0 * 100.0, 
        "A_s": 2.083e-9, 
        "N_ur": 3.044, 
        "N_ncdm": 0.0,  
        "tau_reio": 0.066, 
        "n_s": ns
})

M.set({"output": "mPk", "P_k_max_1/Mpc": 10.0 * h0, "z_max_pk": 0.0})    
M.compute()
    
Pknormal = np.array([M.pk_lin(ki, 0.0) /(h0 * h0 * h0) for ki in ks_fid]) 

###################################################################################################

arr_v1 = np.array([1.044, 2.044, 3.044, 4.044, 5.044, 6.044])

arrn_v2 = np.array([1.0, 2.0, 3.0, 3.0, 3.0, 3.0, 3.0])
arr_v2 = np.array([0.031, 0.018, 0.0044, 1.0044, 2.0044, 3.0044])


for i in np.arange(len(arr_v1)):

    M.set({
        "omega_b": ob * h0 * h0, 
        "omega_cdm": (om-ob) * h0 * h0, 
        "H0": h0 * 100.0, 
        "A_s": 2.083e-9, 
        "N_ur": arr_v1[i], 
        "N_ncdm": 0.0,  
        "tau_reio": 0.066, 
        "n_s": ns
    })

    M.set({"output": "mPk", "P_k_max_1/Mpc": 10.0 * h0, "z_max_pk": 0.0})
    
    M.compute()
    
    Pk = np.array([M.pk_lin(ki, 0.0) /(h0 * h0 * h0) for ki in ks_fid]) 

    plt.loglog(ks_fid, Pk/Pknormal, label = r'$N_{\mathrm{eff}}$ = %.3f (m1)' % M.Neff())
    
    M.set({
        "omega_b": ob * h0 * h0, 
        "omega_cdm": (om-ob) * h0 * h0, 
        "H0": h0 * 100.0, 
        "A_s": 2.083e-9, 
        "N_ur": arr_v2[i], 
        "N_ncdm": arrn_v2[i],  
        "tau_reio": 0.066, 
        "n_s": ns
    })

    M.set({"output": "mPk", "P_k_max_1/Mpc": 10.0 * h0, "z_max_pk": 0.0})
    
    M.compute()
    
    Pk = np.array([M.pk_lin(ki, 0.0) /(h0 * h0 * h0) for ki in ks_fid]) 

    plt.loglog(ks_fid, Pk/Pknormal, label = r'$N_{\mathrm{eff}}$ = %.3f (m2)' % M.Neff(), linestyle='--')

#ax1.set_title(r'$N_{\mathrm{eff}} = \Delta N_{\mathrm{ur}}$')
plt.legend()

plt.show() 

###################################################################################################

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
from classy import Class 
M = Class()

ob=0.04814
ns=0.97 
om = 0.31
h0 = 0.676
ks_fid = np.logspace(np.log10(0.001 * h0), np.log10(10.0 * h0), 100, base=10) # ks in 1/ MPC unit

# power spectrum to normalize 

M.set({
        "omega_b": ob * h0 * h0, 
        "omega_cdm": (om-ob) * h0 * h0, 
        "H0": h0 * 100.0, 
        "A_s": 2.083e-9, 
        "N_ur": 3.044, 
        "N_ncdm": 0.0,  
        "tau_reio": 0.066, 
        "n_s": ns
})

M.set({"output": "mPk", "P_k_max_1/Mpc": 10.0 * h0, "z_max_pk": 0.0})    
M.compute()
    
Pknormal = np.array([M.pk_lin(ki, 0.0) /(h0 * h0 * h0) for ki in ks_fid]) 

###################################################################################################
arr_v2 = np.array([0.01, 1.0, 2.0])

for i in np.arange(len(arr_v2)):

    M.set({
        "omega_b": ob * h0 * h0, 
        "omega_cdm": (om-ob) * h0 * h0, 
        "H0": h0 * 100.0, 
        "A_s": 2.083e-9, 
        "N_ur": arr_v2[i], 
        "N_ncdm": 1.0,  
        "tau_reio": 0.066, 
        "n_s": ns
    })

    M.set({"output": "mPk", "P_k_max_1/Mpc": 10.0 * h0, "z_max_pk": 0.0})
    
    M.compute()
    
    Pk = np.array([M.pk_lin(ki, 0.0) /(h0 * h0 * h0) for ki in ks_fid]) 

    plt.loglog(ks_fid, Pk/Pknormal, label = r'$N_{\mathrm{eff}}$ = %.3f (m1)' % M.Neff())
    
plt.legend()

plt.show() 

## Testing varying Beta with CLASS and CAMB

In [ ]:
import sys 
sys.path.append("../")
#sys.path.append("../../")
import barry 
import numpy as np
import matplotlib.pyplot as plt 

from barry.models import PowerBeutler2017

model1 = PowerBeutler2017(
    isotropic=True,
    vary_phase_shift_neff=True,
    #poly_poles=[0],
    dilate_smooth=False,
    n_poly=6, 
) # setting up choice for BAO model - isotropic model for plotting - this model uses CAMB by default 

model2 = PowerBeutler2017(
    isotropic=True,
    vary_phase_shift_neff=True,
    #poly_poles=[0],
    dilate_smooth=False,
    n_poly=6,
    use_classorcamb='CLASS'
) # setting up choice for BAO model - isotropic model for plotting - this model uses CLASS 


cosmology = {
            "om": 0.31,
            "h0": 0.67,
            "z":  0.51,
            "ob": 0.048,
            "ns": 0.97,
            "mnu": 0.0,
            "reconsmoothscale": 15,
            "Neff": 3.044,
        }
model1.set_cosmology(cosmology) # initialises self.camb object 
model2.set_cosmology(cosmology) # initialises self.camb object 

ks1 = model1.camb.ks
ks2 = model2.camb.ks
params_dict = {j.name: j.default for j in model1.params}

ax1 = plt.subplot(121)
ax2 = plt.subplot(122)

ax2.set_xlim(5e-4, 0.5)
ax1.set_xlim(5e-4, 10.0)

params_dict["beta_phase_shift"] = 1.0
k, pk1 = model1.compute_power_spectrum(ks1, params_dict, smooth=True)

for phase_shift_parameter in [0.0, 1.0, 2.0]:

    params_dict["beta_phase_shift"] = phase_shift_parameter
    k, pk = model1.compute_power_spectrum(ks1, params_dict)
    
    print(model1.camb.get_data()["r_s"])
    
    ax1.loglog(k, pk[0], label=r"$\beta(N_{\mathrm{eff}})$ = %.3f, M1" % phase_shift_parameter)
    ax2.semilogx(k, pk[0]/pk1[0], label=r"$\beta(N_{\mathrm{eff}})$ = %.3f, M1" % phase_shift_parameter, alpha=0.9)
    
    k, pk = model2.compute_power_spectrum(ks2, params_dict)
    
    ax1.loglog(k, pk[0], label=r"$\beta(N_{\mathrm{eff}})$ = %.3f, M2" % phase_shift_parameter, linestyle= '-.')
    ax2.semilogx(k, pk[0]/pk1[0], label=r"$\beta(N_{\mathrm{eff}})$ = %.3f, M2" % phase_shift_parameter, alpha=0.9, linestyle= '-.')
    
    print(model2.camb.get_data()["r_s"])
    
print("Beutler et al 2017 power spectrum model, recomputed with varying Neff phase shift parameter beta")
ax1.set_title('isotropic monopole')
ax2.set_title(r'ratio isotropic monopole to model with $N_{\mathrm{eff}}$=3.044 and no wiggles ($\beta$=1)')
plt.subplots_adjust(wspace=0.1, left=0, right=2, top=1)
ax1.legend()
ax2.legend()
plt.show() 

## Varying power spectrum Neff with CLASS vs CAMB 

In [ ]:
from barry.models import PowerBeutler2017

# Set up the appropriate model for the power spectrum

modelcamb = PowerBeutler2017(
    vary_neff=True,
    isotropic=True,
    n_poly=6,
) # setting up choice for BAO model 

modelclass = PowerBeutler2017(
    vary_neff=True,
    isotropic=True,
    n_poly=6,
    use_classorcamb='CLASS'
)

cosmology = {
            "om": 0.31,
            "h0": 0.67,
            "z":  0.51,
            "ob": 0.048,
            "ns": 0.97,
            "mnu": 0.0,
            "reconsmoothscale": 15,
            "Neff": 3.044,
        }
modelcamb.set_cosmology(cosmology) # initialises self.camb object 
kcamb = modelcamb.camb.ks

modelclass.set_cosmology(cosmology)
kclass = modelclass.camb.ks

# modelcamb.camb._generate_data()
#modelclass.camb._generate_data() # - only needs to be run one time to calculate power spectrum at varying Neff 


ax1 = plt.subplot(121)
ax2 = plt.subplot(122)

#ax1.set_xlim(2e-4, 100)
#ax2.set_xlim(2e-4, 100)

ax1.set_ylim(5e-5, 1e6)
ax2.set_ylim(-0.5, 3)


ax1.set_title("Matter power spectrum")
ax2.set_title(r"Matter power spectrum ratio to $N_{\mathrm{eff}}$=3.044 model")

pk_3044 = modelcamb.camb.get_data(cosmology["om"], Neff=3.044)

for i in [1.044, 3.044, 5.044]:
    
    pkcamb = modelcamb.camb.get_data(cosmology["om"], Neff=i) # interpolates for desired Neff and returns power spectrum data 
    # (only works as long as self.cab._generate_data() has been run) 
    
    ax1.loglog(modelcamb.camb.ks, pkcamb['pk_lin_0'], label=r"$N_{\mathrm{eff}}$ = %.2f, M1" % i)
    ax2.semilogx(modelcamb.camb.ks, pkcamb['pk_lin_0']/pk_3044['pk_lin_0'], label="$N_{\mathrm{eff}}$ = %.2f, M1" % i)
    
    pkclass = modelclass.camb.get_data(cosmology["om"], Neff=i) # interpolates for desired Neff and returns power spectrum data 
    # (only works as long as self.cab._generate_data() has been run) 
    
    ax1.loglog(modelclass.camb.ks, pkclass['pk_lin_0'], label=r"$N_{\mathrm{eff}}$ = %.2f, M2" % i, linestyle='-.')
    ax2.semilogx(modelclass.camb.ks, pkclass['pk_lin_0']/pk_3044['pk_lin_0'], label="$N_{\mathrm{eff}}$ = %.2f, M2" % i, linestyle='-.')
    
    
plt.subplots_adjust(wspace=0.4, left=0, right=1.7)    
ax2.set_ylim([0.5, 2.5])
ax1.legend()
ax2.legend()

## testing plotting the correlation function 

In [ ]:
import sys 
sys.path.append("../")
import barry 
import numpy as np
import matplotlib.pyplot as plt 

from barry.models import CorrBeutler2017
from barry.cosmology.pk2xi import *#pk2xi

In [ ]:
# Set up the appropriate model for the power spectrum
model = CorrBeutler2017(
    vary_neff=True,
    isotropic=True,
) # setting up choice for BAO model 

cosmology = {
            "om": 0.31,
            "h0": 0.67,
            "z":  0.51,
            "ob": 0.048,
            "ns": 0.97,
            "mnu": 0.0,
            "reconsmoothscale": 15,
            "Neff": 3.044,
        }
model.parent.set_cosmology(cosmology) # initialises self.camb object 
dat = model.parent.camb.get_data()
#print(model.parent.fix_params)
k, pk = dat["ks"], dat["pk_lin_0"]

model.pk2xi_0 = PowerToCorrelationGauss(k, ell=0)
#model.pk2xi_0 = PowerToCorrelationSphericalBessel(ell=0)

ax1 = plt.subplot(111)

dists = np.linspace(0, 200, 200)
s, xi = dists, model.pk2xi_0(k, pk, dists)

#plt.plot(s, xi* (s**2))

#s, xi, poly = model.compute_correlation_function(dists, params_dict, smooth=True, vary_neff=True)

for neff in [1.0, 2.0, 3.0, 3.044, 4.0, 5.0]:

    dat = model.parent.camb.get_data(Neff=neff, om=cosmology["om"], h0=cosmology["h0"])
    
    k1, pk1 = dat["ks"], dat["pk_lin_0"]
    
    xi_i = model.pk2xi_0(k1, pk1, s)
    ax1.plot(s, xi_i* (s**2), label=r"$N_{\mathrm{eff}}$ = %.3f" % neff)


print("Beutler et al 2017 power spectrum model to BAO, recomputed with varying Neff")
ax1.set_title("Correlation function with varying Neff")
ax1.set_xlabel(r'$s$ $\mathrm{Mpc}$ $h^{-1}$')
ax1.set_ylabel(r'$s^2 \xi(s)$ $\mathrm{Mpc}^2$ $h^{-2}$')
ax1.legend()
plt.show()


## now plot monopole, quadrupole with varying Neff

In [ ]:
# Set up the appropriate model for the power spectrum
model = CorrBeutler2017(
    vary_neff=True,
    isotropic=False 
) # setting up choice for BAO model 

cosmology = {
            "om": 0.31,
            "h0": 0.67,
            "z":  0.51,
            "ob": 0.048,
            "ns": 0.97,
            "mnu": 0.0,
            "reconsmoothscale": 15,
            "Neff": 3.044,
        }
model.parent.set_cosmology(cosmology) # initialises self.camb object 
dat = model.parent.camb.get_data()
k, pk = dat["ks"], dat["pk_lin_0"]

model.pk2xi_0 = PowerToCorrelationGauss(k, ell=0)
model.pk2xi_2 = PowerToCorrelationGauss(k, ell=2)
model.pk2xi_4 = PowerToCorrelationGauss(k, ell=4)

params_dict = {j.name: j.default for j in model.params}

ax1 = plt.subplot(131)
ax2 = plt.subplot(132)
ax3 = plt.subplot(133)

params_dict["Neff"] = 3.044
params_dict['beta'] = 0.525

dists = np.linspace(1, 200, 200)


for neff in [1.0, 2.0, 3.0, 3.044, 4.0, 5.0]:
    
    params_dict["Neff"] = neff
    #print(params_dict)
    s, xi = model.compute_correlation_function(dists, params_dict, vary_neff=True)
    ax1.plot(s, xi[0] * (s**2), label=r"$N_{\mathrm{eff}}$ = %.3f" % neff)
    ax2.plot(s, xi[1] * (s**2), label=r"$N_{\mathrm{eff}}$ = %.3f" % neff)
    ax3.plot(s, xi[2] * (s**2), label=r"$N_{\mathrm{eff}}$ = %.3f" % neff)

print("Beutler et al 2017 power spectrum model to BAO multipoles, recomputed with varying Neff")
ax1.set_title("Correlation monopole with varying Neff")
ax2.set_title("Correlation quadrupole with varying Neff")
ax3.set_title("Correlation hexadecapole with varying Neff")

ax1.set_xlabel(r'$s$ $\mathrm{Mpc}$ $h^{-1}$')
ax2.set_xlabel(r'$s$ $\mathrm{Mpc}$ $h^{-1}$')
ax3.set_xlabel(r'$s$ $\mathrm{Mpc}$ $h^{-1}$')

ax1.set_ylabel(r'$s^2 \xi(s)$ $\mathrm{Mpc}^2$ $h^{-2}$')

ax1.legend()
ax2.legend()
ax3.legend()
plt.subplots_adjust(wspace=0.2, left=0, right=2.0)    
plt.show()

## code to plot monopole, quadrupole with varying phase shift 

In [ ]:
# Set up the appropriate model for the power spectrum
model = CorrBeutler2017(
    vary_phase_shift_neff=True,
    isotropic=False 
) # setting up choice for BAO model 

cosmology = {
            "om": 0.31,
            "h0": 0.67,
            "z":  0.51,
            "ob": 0.048,
            "ns": 0.97,
            "mnu": 0.0,
            "reconsmoothscale": 15,
            "Neff": 3.044,
        }
model.parent.set_cosmology(cosmology) # initialises self.camb object 
dat = model.parent.camb.get_data()
k, pk = dat["ks"], dat["pk_lin_0"]

model.pk2xi_0 = PowerToCorrelationGauss(k, ell=0)
model.pk2xi_2 = PowerToCorrelationGauss(k, ell=2)
model.pk2xi_4 = PowerToCorrelationGauss(k, ell=4)

params_dict = {j.name: j.default for j in model.params}

ax1 = plt.subplot(131)
ax2 = plt.subplot(132)
ax3 = plt.subplot(133)
    
params_dict['beta'] = 0.525

dists = np.linspace(1, 200, 200)

for phase_shift_parameter in [0., 1.0, 2.0, 3.0]:
    
    params_dict["beta_phase_shift"] = phase_shift_parameter
    s, xi = model.compute_correlation_function(dists, params_dict)
    ax1.plot(s, xi[0] * (s**2), label=r"$\beta(N_{\mathrm{eff}})$ = %.3f" % phase_shift_parameter)
    ax2.plot(s, xi[1] * (s**2), label=r"$\beta(N_{\mathrm{eff}})$ = %.3f" % phase_shift_parameter)
    ax3.plot(s, xi[2] * (s**2), label=r"$\beta(N_{\mathrm{eff}})$ = %.3f" % phase_shift_parameter)

print("Beutler et al 2017 power spectrum model to BAO multipoles, recomputed with varying phase shift due to Neff")
ax1.set_title(r"Correlation monopole with varying $\beta(N_{\mathrm{eff}})$")
ax2.set_title(r"Correlation quadrupole with varying $\beta(N_{\mathrm{eff}})$")
ax3.set_title(r"Correlation hexadecapole with varying $\beta(N_{\mathrm{eff}})$")

ax1.set_xlabel(r'$s$ $\mathrm{Mpc}$ $h^{-1}$')
ax2.set_xlabel(r'$s$ $\mathrm{Mpc}$ $h^{-1}$')
ax3.set_xlabel(r'$s$ $\mathrm{Mpc}$ $h^{-1}$')

ax1.set_ylabel(r'$s^2 \xi(s)$ $\mathrm{Mpc}^2$ $h^{-2}$')

ax1.legend()
ax2.legend()
ax3.legend()
plt.subplots_adjust(wspace=0.2, left=0, right=2.0)    
plt.show()